In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import polars as pl

# Turn this on for higher DPI on images
# mpl.rcParams['figure.dpi'] = 300

# Getting started
After downloading `dolt`, I exported the data to CSV (though you could read directly from the databse with `polars` or `pandas`)
```
dolt table export sales sales-2022-03-10.csv
```
Then I did
```
grep -ax '.*' sales-2022-03-10.csv > sales-2022-03-10-cleaned.csv  
```
to get rid of Unicode errors

In [ ]:
dtypes = {
    'property_id': str,
    'zip5': str,
}


lazydf = pl.scan_csv('sales-2022-03-10-cleaned.csv', dtypes = dtypes, encoding = 'utf8-lossy')

In [ ]:
def outlier_index() -> pl.Expr:
    """
    We want to create an index for each sale to know if it's an outlier.
    This is the ratio of the highest price to the lowest price.
    """
    
    return (pl.col('sale_price').sort().last()/pl.col('sale_price').sort().first()).over('property_id')

## Filter main dataframe down

In [ ]:
selected_property_types = 'dwelling|home|condo|family|residence|residential|apartment|homesite|duplex|triplex|fourplex|resid|apt'

df = (
    lazydf
    .filter(pl.col('property_type').str.to_lowercase().str.contains(selected_property_types))
    .filter(pl.col('sale_price') > 20_000)
    
    # remove weird address
    .filter(~pl.col('physical_address').str.contains('^0 |^- '))
    .filter(pl.col('physical_address').str.lengths() > 5)
    
    .with_column(
        pl.col('sale_date').str.strptime(pl.Datetime, fmt = "%Y-%m-%d %H:%M:%S %z %Z", strict = False)
    )
    .with_columns(
        [
            pl.col('state').cast(pl.Categorical),
            pl.col('sale_date').dt.year().alias('sale_year'),
            pl.col('sale_date').dt.month().alias('sale_month'),            
            pl.col('sale_date').dt.day().alias('sale_day'),
        ]
    )
    
    # sanity check -- some houses have weird sale years
    .filter(pl.col('sale_year') < 2022)
    
    .with_columns(
        [
            (pl.col('sale_year') - pl.col('year_built')).alias('property_age'),
            outlier_index().alias('outlier_index'),
            pl.lit(1).count().over('physical_address').alias('property_count'),
            pl.lit(1).count().over(['property_id', 'physical_address']).alias('times_sold'),   
            pl.col('sale_price').max().over('property_id').alias('sale_max'),
            pl.col('sale_price').min().over('property_id').alias('sale_min'),
            pl.col('sale_price').median().over(['physical_address', 'property_id']).alias('sale_median'),
        ]
    )
    .with_columns(
        [
            pl.max([pl.col('sale_price')/pl.col('sale_median'),pl.col('sale_median')/pl.col('sale_price')]).alias('outlier_index')
        ]
    )
    
    # not needed for this analysis
    .drop(['source_url', 'book', 'page'])
    .collect()
)

## `predata` is a dataframe which holds some prefiltered data

After loading the entire dataframe into `df`, I wanted slightly more control for testing and plotting purposes. So I made a pre-filtered dataframe called `predata`, big enough that I could do the whole analysis with it, but not so big that the plots took forever to run.

In [ ]:
predata = (
    # comment out the sample if you want the complete data
    df.sample(frac = .01)
    
    # we don't have complete records for these throughout the last decade
    # this skews the relationship of sale_date to sale
    .filter(pl.col('state') != 'WI')
    .filter(pl.col('state') != 'NC')
    
    # remove any properties that appear more than 100 times as the same address
    .filter(pl.col('property_count') < 100)
    
    # ...with "age" < 0
    .filter(pl.col('property_age') >= 0)
    
    # ...where the min/max scores differ by too much
    .filter(pl.col('outlier_index') < 3)
    
    # ...where the sale_year is too old
    .filter(pl.col('sale_year') > 1975)
)

### Scatterplot of all sales

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))

data = predata.clone()

xs = [np.datetime64(t) for t in data['sale_date']]
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .01, alpha = .01)

yticks = {0:         '$0',
          100_000:   '$100k',
          200_000:   '$200k',
          500_000:   '$500k',
          1_000_000: '$1M',
          1_500_000: '$1.5M',
          2_000_000: '$2M',}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title('Residential property sales in the US')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(16)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 13)
ax.yaxis.set_tick_params(size = 13)

plt.savefig('images/sales_total.png', facecolor = 'white')
plt.show()

### New-house premium

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))

max_age = 25

data = (
    predata
    .filter(pl.col('property_age') <= max_age)
    .with_column(
         pl.col('sale_price').mean().over('sale_year').alias('mean_price_per_year')
     )
    .filter(pl.col('sale_price') < pl.col('mean_price_per_year')*40)
    .groupby(['property_age', 'sale_year'])
    .agg([pl.col('sale_price').mean(), pl.col('mean_price_per_year').first().alias('ppy')])
    .with_column(
         (pl.col('sale_price_mean')/pl.col('ppy')).alias('ratio')
     )[['property_age', 'ratio']]
    .groupby('property_age')      
    .agg(pl.col('ratio').mean())      
    .sort('property_age')
)

xs, ys = data

ax.set_title('The new-house premium: how do house prices depreciate with age?')
ax.set_ylabel('premium (%)')
ax.set_xlabel('property age')

ax.set_ylim([.6, 1.4])
ax.set_xlim([0, max_age])

ax.title.set_size(16)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 13)
ax.yaxis.set_tick_params(size = 13)

ax.step(np.asarray(xs), np.asarray(ys))

plt.savefig('images/new_house_premium.png', facecolor = 'white')
plt.show()

### Finding appreciation rates

In [ ]:
from datetime import datetime
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * np.exp(-b * x)

fig, ax = plt.subplots(figsize = (12, 6))

max_median = 750_000

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('sale_median') < max_median)
    # .filter(pl.col('sale_median') > 25_000)
)

xs = [np.datetime64(t) for t in data['sale_date']]
# ordinals are needed to create a fit line
xs_ord = np.asarray([t.toordinal() for t in data['sale_date']])
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .2,  alpha = .02)

def func(x, a, b):
    return a * np.exp(b * x)

xs_tmp = xs_ord - 730_000
xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
ys_tmp = ys/1000

p0 = [350, .001]

popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)

ax.plot([np.datetime64(datetime.fromordinal(int(a))) for a in xs_plt],
        [1000*func(x-730_000, *popt) for x in xs_plt], 
        color = 'red', 
        label = f'appreciation approx. {round(popt[1]*100*365, 1)}%/year')

yticks = {100_000:   '$100k',
          250_000:   '$250k',
          500_000:   '$500k',
          1_000_000: '$1M',}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title(f'Rising property values from 2010 to 2020, median_sale < ${max_median}k')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 13)

plt.savefig('images/appreciation_total_less_than_750k.png', facecolor = 'white')
plt.show()

In [ ]:
import calendar
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * np.exp(-b * x)

fig, ax = plt.subplots(figsize = (12, 6))

max_median = 750_000
min_median = 250_000

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('sale_median') < max_median)
    .filter(pl.col('sale_median') > min_median)
    .filter(pl.col('times_sold') > 1)
    # .filter(pl.col('sale_median') > 25_000)
)

xs = [np.datetime64(t) for t in data['sale_date']]
# ordinals are needed to create a fit line
xs_ord = np.asarray([t.toordinal() for t in data['sale_date']])
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .05,  alpha = .1)

def func(x, a, b):
    return a * np.exp(b * x)

xs_tmp = xs_ord - 730_000
xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
ys_tmp = ys/1000

p0 = [350, .001]

popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)

ax.plot([np.datetime64(datetime.fromordinal(int(a))) for a in xs_plt],
        [1000*func(x-730_000, *popt) for x in xs_plt], 
        color = 'red', 
        label = f'appreciation approx. {round(popt[1]*100*365, 1)}%/year')

yticks = {100_000:   '$100k',
          250_000:   '$250k',
          500_000:   '$500k',
          1_000_000: '$1M',}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title(f'Rising property values from 2010 to 2020, \$250k < median_sale < \$750k')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 13)

plt.savefig('images/appreciation_total_middle_median.png', facecolor = 'white')
plt.show()

In [ ]:
from datetime import datetime
import calendar
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * np.exp(-b * x)

fig, ax = plt.subplots(figsize = (12, 6))

max_median = 1_500_000
min_median = 750_000

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('sale_median') < max_median)
    .filter(pl.col('sale_median') > min_median)
    .filter(pl.col('times_sold') > 1)
    # .filter(pl.col('sale_median') > 25_000)
)

xs = [np.datetime64(t) for t in data['sale_date']]
# ordinals are needed to create a fit line
xs_ord = np.asarray([t.toordinal() for t in data['sale_date']])
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .2,  alpha = .2)

def func(x, a, b):
    return a * np.exp(b * x)

xs_tmp = xs_ord - 730_000
xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
ys_tmp = ys/1000

p0 = [350, .001]

popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)

ax.plot([np.datetime64(datetime.fromordinal(int(a))) for a in xs_plt],
        [1000*func(x-730_000, *popt) for x in xs_plt], 
        color = 'red', 
        label = f'Prices increase at about {round(popt[1]*100*365, 1)}%/year')

yticks = {
    0: '$0',
    500_000:   '$500k',
    1_000_000: '$1M',
    2_000_000: '$2M',
    3_000_000: '$3M',
}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title(f'Rising property values from 2010 to 2020, \$750k < median_sale < \$1.5M')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 13)

plt.savefig('images/appreciation_total_high_median.png', facecolor = 'white')
plt.show()

In [ ]:
from datetime import datetime
import calendar
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * np.exp(-b * x)

fig, ax = plt.subplots(figsize = (12, 6))

max_median = 3_000_000
min_median = 1_500_000

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('sale_median') < max_median)
    .filter(pl.col('sale_median') > min_median)
    .filter(pl.col('times_sold') > 1)
    # .filter(pl.col('sale_median') > 25_000)
)

xs = [np.datetime64(t) for t in data['sale_date']]
# ordinals are needed to create a fit line
xs_ord = np.asarray([t.toordinal() for t in data['sale_date']])
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .2,  alpha = .5)

def func(x, a, b):
    return a * np.exp(b * x)

xs_tmp = xs_ord - 730_000
xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
ys_tmp = ys/1000

p0 = [350, .001]

popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)

ax.plot([np.datetime64(datetime.fromordinal(int(a))) for a in xs_plt],
        [1000*func(x-730_000, *popt) for x in xs_plt], 
        color = 'red', 
        label = f'Prices increase at about {round(popt[1]*100*365, 1)}%/year')

yticks = {
    0: '$0',
    1_000_000: '$1M',
    3_000_000: '$3M',
    5_000_000: '$5M',
}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title(f'Rising property values from 2010 to 2020, \$1.5M < median_sale < \$3M')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 13)

plt.savefig('images/appreciation_total_very_high_median.png', facecolor = 'white')

plt.show()

In [ ]:
from datetime import datetime
import calendar
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * np.exp(-b * x)

fig, ax = plt.subplots(figsize = (12, 6))

max_median = 400_000
min_median = 100_000

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('sale_median') < max_median)
    .filter(pl.col('sale_median') > min_median)
    .filter(pl.col('times_sold') > 1)
    # .filter(pl.col('sale_median') > 25_000)
)

xs = [np.datetime64(t) for t in data['sale_date']]
# ordinals are needed to create a fit line
xs_ord = np.asarray([t.toordinal() for t in data['sale_date']])
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .05,  alpha = .1)

def func(x, a, b):
    return a * np.exp(b * x)

xs_tmp = xs_ord - 730_000
xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
ys_tmp = ys/1000

p0 = [350, .001]

popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)

ax.plot([np.datetime64(datetime.fromordinal(int(a))) for a in xs_plt],
        [1000*func(x-730_000, *popt) for x in xs_plt], 
        color = 'red', 
        label = f'Prices increase at about {round(popt[1]*100*365, 1)}%/year')

yticks = {
    0: '$0',
    100_000: '$100k',
    300_000: '$300k',
    500_000: '$500k',
    700_000: '$600k',
}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title(f'Rising property values from 2010 to 2020, \$100k < median_sale < \$400k')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 13)
plt.savefig('images/appreciation_total_low_median.png', facecolor = 'white')

plt.show()

### Cheapest month to buy

In [ ]:
import calendar
fig, ax = plt.subplots(figsize = (12, 6))

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('sale_median') < max_median)
)

start_date = datetime(2011, 1, 1, 0, 0)

data['adjusted_sale_price'] = pl.Series(data.apply(lambda row: row[8]*np.exp((start_date - row[6]).days*popt[1]))['apply'])

xs, ys = (
    data
    .groupby('sale_month')
    .agg(
        [
            pl.col('adjusted_sale_price').mean()
        ]
    )
    .sort('sale_month')
)

ax.bar(xs, ys)

yticks = {val: f'${val//1_000}k' for val in range(220_000, 280_000, 10_000)}

plt.title("What's the cheapest month to buy a house?")
ax.set_ylabel('adjusted sale price [USD]')
ax.set_xlabel('sale month')

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_xlim([0, 13])
ax.set_ylim([ys.min()/1.1, ys.max()*1.05])

ax.set_xticks([i for i in range(1,12+1)])
ax.set_xticklabels([calendar.month_abbr[i] for i in ax.get_xticks()])
ax.xaxis.set_tick_params(rotation = 10)

ax.title.set_size(20)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

plt.savefig('images/cheapest_month_to_buy.png', facecolor = 'white')

plt.show()

### Sales counts over time

In [ ]:
fig, ax = plt.subplots(figsize = (16, 6))

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .groupby('sale_date')
    .agg(
        [
            pl.lit(1).count()
        ]
    )
    .sort('sale_date')
)

xs = [np.datetime64(t) for t in data['sale_date']]
ys = np.asarray(data['literal_count'])

def moving_average(a, n = 3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

window = 60
xs = xs[window - 1:]
ys = moving_average(ys, n = window)

plt.title('Number of property sales over time (2 month moving average)')
plt.xlabel('year')
plt.ylabel('num. property sales [avg]')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([0, ys.max()*1.2])

ax.title.set_size(16)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.plot(xs, ys)

plt.savefig('images/property_sales_over_time.png', facecolor = 'white')

plt.show()

### Sales counts over time by state

In [ ]:
data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .groupby(['sale_date', 'state'])
    .agg(
        [
            pl.lit(1).count()
        ]
    )
    .sort('sale_date')
)

In [ ]:
fig, ax = plt.subplots(figsize = (16, 6))

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .groupby(['sale_date', 'state'])
    .agg(
        [
            pl.lit(1).count()
        ]
    )
    .sort('sale_date')
)

states = data['state'].unique()

def moving_average(a, n = 3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

for i, state in enumerate(states):

    data2 = data.filter(pl.col('state') == state)
    
    window = 60

    xs = [np.datetime64(t) for t in data2['sale_date']]
    ys = np.asarray(data2['literal_count'])
    
    if i == 0:
        globxmin = min(xs)
        globxmax = max(xs)
        
    else: 
        if min(xs) < globxmin:
            globxmin = min(xs)

        if max(xs) > globxmax:
            globxmax = max(xs)

    xs = xs[window - 1:]
    ys = moving_average(ys, n = window)

    ax.plot(xs, ys, alpha = 1, label = state)

plt.title('Property sales by state (2 month moving average)')
plt.xlabel('year')
plt.ylabel('num. property sales [avg]')

ax.set_xlim([globxmin, globxmax])
ax.set_ylim([0, 600])

ax.title.set_size(16)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

plt.legend()

plt.savefig('images/property_sales_by_state.png', facecolor = 'white')

plt.show()

### Rurality score

In [ ]:
_fips_rurality = pl.read_csv('fips_rurality.csv', dtypes = {'FIPS': str})
_zips_fips_conv = pl.read_csv('zip_fips_conversion.csv', dtypes = {'ZIP': str, 'STCOUNTYFP': str})
zip_info = _zips_fips_conv.join(_fips_rurality, left_on = 'STCOUNTYFP', right_on = 'FIPS')

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))

data = (
    predata.filter(pl.col('sale_year') > 2000).filter(pl.col('sale_price') < 5_000_000)
).join(zip_info[['ZIP', 'RUCC_2013']], left_on = 'zip5', right_on = 'ZIP')

urban_score_split = 1

data1 = data.filter(pl.col('RUCC_2013') > urban_score_split).groupby('sale_date').agg([pl.col('sale_price').mean()]).sort('sale_date')
data2 = data.filter(pl.col('RUCC_2013') <= urban_score_split).groupby('sale_date').agg([pl.col('sale_price').mean()]).sort('sale_date')

xs1 = [np.datetime64(t) for t in data1['sale_date']]
ys1 = np.asarray(data1['sale_price_mean'])

xs2 = [np.datetime64(t) for t in data2['sale_date']]
ys2 = np.asarray(data2['sale_price_mean'])

def moving_average(a, n = 3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

window = 120
xs1 = xs1[window - 1:]
ys1 = moving_average(ys1, n = window)

xs2 = xs2[window - 1:]
ys2 = moving_average(ys2, n = window)

ax.step(xs1, ys1, c = 'r', label = 'more rural')
ax.step(xs2, ys2, c = 'b', label = 'more urban')

yticks = {100_000: '$100k',
          250_000: '$250k',
          500_000: '$500k',
          750_000: '$750k',
          1_000_000: '$1M',}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title('Sale prices by category, 2010-2020')
ax.set_ylabel('mean sale price by category [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs2), max(xs2)])
ax.set_ylim([0, 1_000_000])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 15)

plt.savefig('images/sale_prices_by_category.png', facecolor = 'white')
plt.show();

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

cmap = cm.autumn
norm = Normalize(vmin=0, vmax=7)

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))

data = (
    predata.filter(pl.col('sale_year') > 2000).filter(pl.col('sale_price') < 5_000_000)
).join(zip_info[['ZIP', 'RUCC_2013']], left_on = 'zip5', right_on = 'ZIP')

def moving_average(a, n = 3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

for i in range(0,7):
    data1 = data.filter(pl.col('RUCC_2013') == i).groupby('sale_date').agg([pl.col('sale_price').mean()]).sort('sale_date')

    xs1 = [np.datetime64(t) for t in data1['sale_date']]
    ys1 = np.asarray(data1['sale_price_mean'])

    window = 120
    xs1 = xs1[window - 1:]
    ys1 = moving_average(ys1, n = window)

    ax.step(xs1, ys1, c= cmap(norm(i)), label = f'{i}')


yticks = {100_000: '$100k',
          250_000: '$250k',
          500_000: '$500k',
          750_000: '$750k',
          1_000_000: '$1M',}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title('Sale prices by RUCC rurality score (0 is most urban, 6 is most rural), 2010-2020')
ax.set_ylabel('mean sale price by category [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs1), max(xs1)])
ax.set_ylim([0, 1_000_000])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 15)

plt.savefig('images/sale_prices_by_category_detailed.png', facecolor = 'white')
plt.show();

### Finding the most common buyers

In [ ]:
buyers = (df.
    filter(pl.col('buyer_name') != '')
    .with_column(pl.lit(1).alias('counts'))
    .groupby(['buyer_name', 'sale_year'])
    .agg(pl.col('counts').count())
    .sort('counts_count')
)

In [ ]:
for year in [2017, 2018, 2019, 2020, 2021]:
    print(buyers.filter(pl.col('sale_year') == year).sort('counts_count').tail(3))

In [ ]:
print(((buyers
 .filter(pl.col('sale_year') > 2015)
 .groupby(['sale_year', 'buyer_name'])
 .agg(pl.col('buyer_name').count())
 .groupby('sale_year')
 .apply(lambda x: x.sort('buyer_name_count', reverse = False).tail(3))    
 .sort('sale_year')
)))

### ZIP Code map

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data as vegadata

zipsource = pl.DataFrame(pd.read_csv(vegadata.zipcodes.url, dtype = {'zip_code': str}))

In [ ]:
data = predata.join(zipsource, left_on = 'zip5', right_on = 'zip_code')

In [ ]:
dt = (data
      .groupby(['zip5', 'latitude', 'longitude'])
      .agg(pl.count())
      .with_column(
          np.log10(pl.col('count')).cast(int).alias('logct')
      )
      .to_pandas()
     )

In [ ]:
states = alt.topo_feature(vegadata.us_10m.url, feature='states')

alt.data_transformers.disable_max_rows()

# US states background
background = alt.Chart(states).mark_geoshape(
    fill='white',
    stroke='gray'
).properties(
    width=975,
    height=600
).project('albersUsa')

points = (
    alt.Chart(dt)
    .mark_circle(size=15).encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        color='logct:Q',
        # size = 'logct:N',
        tooltip='zip5:N'
    ).project(
        type='albersUsa'
    ).properties(
        width=975,
        height=600
    )
)

# points.save('zip_distribution.png')
background + points

### LA MAP

In [ ]:
import json

For this part you'll need to download a geoJSON of Los Angeles and manually remove the ZIP Codes for the islands if they exist. You can try here: https://data.lacounty.gov/Geospatial/ZIP-Codes/65v5-jw9f

In [ ]:
with open('la_zips_no_islands.json') as f:
    la_data = json.load(f)

In [ ]:
data_geojson = alt.InlineData(values=la_data, format=alt.DataFormat(property='features',type='json')) 

In [ ]:
la_zips = [x['properties']['zipcode'] for x in data_geojson['values']['features']]

In [ ]:
data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('sale_max') < 10_000_000) # filter out garbage
    .filter(pl.col('sale_max')/pl.col('sale_min') < 4) # filter out outliers
)

In [ ]:
# The chi_sq part of this notebook is kind of experimental

def chi_sq(ys, xs_ord, popt):
    s = 0
    L = len(ys)
    for i in range(L):
        s += (ys[i] - func(xs_ord[i]-730_000, *popt)*1000)**2/L

    return s

def get_zip_code_app_rate(data, zip_code):
    
    data_specific_to_zip = data.filter(pl.col('zip5') == zip_code)
    
    if not data_specific_to_zip: return np.nan

    if len(data_specific_to_zip) < 250: return np.nan

    xs = [np.datetime64(t) for t in data_specific_to_zip['sale_date']]
    xs_ord = np.asarray([t.toordinal() for t in data_specific_to_zip['sale_date']])
    ys = np.asarray(data_specific_to_zip['sale_price'])

    xs_tmp = xs_ord - 730_000
    xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
    ys_tmp = ys/1000

    p0 = [350, 0.0001]

    try:
        popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)
    except (RuntimeError or TypeError):
        return np.nan
    
    if np.sqrt(chi_sq(ys, xs_ord, popt)) > 900_000: return np.nan

    return round(popt[1]*100*365, 1)

In [ ]:
from tqdm import tqdm
app_rates = {}
for i in tqdm(la_zips):
    zip_code = str(i)
    app_rates[zip_code] = get_zip_code_app_rate(data, zip_code)

In [ ]:
zip_app_data = pd.DataFrame({'zipcode': app_rates.keys(), 'rate': app_rates.values()})

In [ ]:
basemap = (
    alt.Chart(data_geojson).mark_geoshape(
        fill='black',
        # stroke='gray',
    )
    .properties(width = 600, height = 600)
    .project('mercator')
)

colors = (
    alt.Chart(data_geojson).mark_geoshape(
        # fill='gray',
        # stroke='gray',
    )
    .encode(
        color = alt.Color('rate:Q', legend=alt.Legend(title="home appreciation rate (%)"))
        
    )
    .transform_lookup(
        lookup = 'properties.zipcode',
        from_ = alt.LookupData(data = zip_app_data, key = 'zipcode', fields = ['rate'])
    )
    .properties(width = 600, height = 600)
    .project('mercator')
)

basemap + colors

In [ ]:
from datetime import datetime
import calendar
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * np.exp(-b * x)

fig, ax = plt.subplots(figsize = (12, 6))

max_median = 400_000
min_median = 100_000

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('zip5') == '90018')
    .filter(pl.col('times_sold') > 1)
    .filter(pl.col('sale_max') < 30_000_000)
    .filter(pl.col('sale_max')/pl.col('sale_min') < 4)
)

xs = [np.datetime64(t) for t in data['sale_date']]
# ordinals are needed to create a fit line
xs_ord = np.asarray([t.toordinal() for t in data['sale_date']])
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .2,  alpha = 1)

def func(x, a, b):
    return a * np.exp(b * x)

xs_tmp = xs_ord - 730_000
xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
ys_tmp = ys/1000

p0 = [300, 0.0001]

popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)

ax.plot([np.datetime64(datetime.fromordinal(int(a))) for a in xs_plt],
        [1000*func(x-730_000, *popt) for x in xs_plt], 
        color = 'red', 
        label = f'appreciation approx. {round(popt[1]*100*365, 1)}%/year')



s = 0
for i in range(len(ys)):
    s += (ys[i] - func(xs_ord[i]-730_000, *popt)*1000)**2/len(ys)

print(np.sqrt(s))

yticks = {
    0: '$0',
    # 100_000: '$100k',
    250_000: '$250k',
    500_000: '$500k',
    # 700_000: '$600k',
    1_000_000: '$1M',
    2_000_000: '$2M',
    # 5_000_000: '$5M',
    # 10_000_000: '$10M',
}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title(f'Rising property values from 2010 to 2020, ZIP code = 90032')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_params(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 13)

plt.savefig('images/appreciation_90032.png', facecolor = 'white')
plt.show()

In [ ]:
from datetime import datetime
import calendar
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * np.exp(-b * x)

fig, ax = plt.subplots(figsize = (12, 6))

max_median = 400_000
min_median = 100_000

data = (
    predata
    .filter((pl.col('sale_year') > 2010) & (pl.col('sale_year') <= 2021))
    .filter(pl.col('zip5') == '90015')
    # .filter(pl.col('times_sold') > 1)
    .filter(pl.col('sale_max') < 10_000_000)
    .filter(pl.col('sale_max')/pl.col('sale_min') < 4)
)

xs = [np.datetime64(t) for t in data['sale_date']]
# ordinals are needed to create a fit line
xs_ord = np.asarray([t.toordinal() for t in data['sale_date']])
ys = np.asarray(data['sale_price'])

ax.scatter(xs, ys, s = .2,  alpha = 1)

def func(x, a, b):
    return a * np.exp(b * x)

xs_tmp = xs_ord - 730_000
xs_plt = list(range(min(xs_tmp + 730_000), max(xs_tmp + 730_000) + 1))
ys_tmp = ys/1000

p0 = [300, 0.0001]

popt, pcov = curve_fit(func, xs_tmp, ys_tmp, p0)

xs_plt_md = [np.datetime64(datetime.fromordinal(int(a))) for a in xs_plt]
ys_plt_md = [1000*func(x-730_000, *popt) for x in xs_plt]

s = 0
for i in range(len(ys)):
    s += (ys[i] - func(xs_ord[i], *popt)*1000)**2

print(s)

ax.plot(xs_plt_md,
        ys_plt_md,
        color = 'red', 
        label = f'appreciation approx. {round(popt[1]*100*365, 1)}%/year')

yticks = {
    0: '$0',
    # 100_000: '$100k',
    250_000: '$250k',
    500_000: '$500k',
    # 700_000: '$600k',
    1_000_000: '$1M',
    2_000_000: '$2M',
    5_000_000: '$5M',
    # 10_000_000: '$10M',
}

ax.set_yticks([*yticks.keys()], [*yticks.values()])

ax.set_title(f'Rising property values from 2010 to 2020, ZIP code = 90404')
ax.set_ylabel('sale price [USD]')
ax.set_xlabel('sale date')

ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(yticks.keys()), max(yticks.keys())])

ax.title.set_size(15)
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
ax.xaxis.set_tick_paddrams(size = 10, labelsize = 15)
ax.yaxis.set_tick_params(size = 10, labelsize = 15)

ax.legend(markerscale = 10, loc = 'upper left', fontsize = 13)

plt.show()